# Pipeline for edge detection of live webcam input
### Convolve with Prewitt opperator


In [ ]:
import numpy as np
import cv2
import torch
from torchvision import transforms
import torch.nn.functional as F

In [ ]:
def gradient(filtered_x, filtered_y):
    return torch.sqrt(torch.multiply(filtered_x,filtered_x) + torch.multiply(filtered_y,filtered_y))

In [ ]:
def conv(img, flat_kernel, kernel_shape, padding, stride):

    output_size = ((img.shape[-2]+padding[0]*2-kernel_shape[0])//stride[0]+1,
                   (img.shape[-1]+padding[1]*2-kernel_shape[1])//stride[1]+1)
    img = img[None, :, :]
    x = F.unfold(img, kernel_size=kernel_shape,
                 stride=stride, padding=padding)

    x = flat_kernel @ x

    return torch.reshape(x, output_size)


In [ ]:
# Use these 1D kernels to construct Prewitt 2D kernels
kernel1 = torch.Tensor([[1, 0, -1]])
kernel2 = torch.Tensor([[1, 0, 1]])

def detect_edges(img, stride=1):

    im_filterx = conv(img, kernel1, (1, 3), (0, 1), (1,stride))[None, :, :]
    im_filterx = conv(im_filterx, kernel2, (3, 1), (1, 0), (stride, 1))

    im_filtery = conv(img, kernel1, (3, 1), (1, 0), (stride, 1))[None, :, :]
    im_filtery = conv(im_filtery, kernel2, (1, 3), (0, 1), (1, stride))

    return gradient(im_filterx, im_filtery)


In [ ]:
cv2.namedWindow("cam edges")
vc = cv2.VideoCapture(0)

if vc.isOpened():  # try to get the first frame
    rval, frame = vc.read()
else:
    rval = False

# Set to integer 1,2,3,4 etc for better performance worse quality
resize_factor = 1
while rval:
    rval, frame = vc.read()
    key = cv2.waitKey(200)
    bw_frame = transforms.Grayscale()(torch.Tensor(frame).permute(2, 0, 1))

    frame_edge = detect_edges(bw_frame, stride=resize_factor)
    frame_edge = (frame_edge*255//torch.max(frame_edge))

    edges_np = frame_edge.numpy().astype(np.uint8)

    colour_edges = cv2.applyColorMap(edges_np, cv2.COLORMAP_TWILIGHT_SHIFTED)


    resized_edges = cv2.resize(
        colour_edges, (frame.shape[1], frame.shape[0]))

    concat = np.concatenate(
        (frame, resized_edges), axis=1).astype(np.uint8)

    cv2.imshow("cam edges", concat)
    
    if key == 27:  # exit on ESC
        break


vc.release()
cv2.destroyWindow("cam edges")
